In [ ]:
import pandas

In [ ]:
data = (pandas.read_csv('data/oslobike/trips-2016.7.1-2016.7.31.csv.zip')
        .append(pandas.read_csv('data/oslobike/trips-2016.8.1-2016.8.31.csv.zip'))
        .append(pandas.read_csv('data/oslobike/trips-2016.9.1-2016.9.30.csv.zip'))
        .append(pandas.read_csv('data/oslobike/trips-2016.10.1-2016.10.31.csv.zip'))
        .append(pandas.read_csv('data/oslobike/trips-2016.11.1-2016.11.30.csv.zip'))
        .append(pandas.read_csv('data/oslobike/trips-2016.6.1-2016.6.30.csv.zip'))
        .append(pandas.read_csv('data/oslobike/trips-2016.5.1-2016.5.31.csv.zip')))

In [ ]:
from dateutil import parser

In [ ]:
data.head()

In [ ]:
from datetime import datetime

In [ ]:
data['timestamp'] = data['Start time'].apply(lambda x: datetime.strptime(x[:-6], '%Y-%m-%d %H:%M:%S'))

In [ ]:
data['timestamp'].head()

In [ ]:
data['count'] = 1

In [ ]:
d = data[['timestamp', 'count']].set_index('timestamp')

In [ ]:
per_hour = d.resample('1h').sum().reset_index().fillna(0)

In [ ]:
per_hour['hour'] = per_hour['timestamp'].apply(lambda x: x.hour)

In [ ]:
per_hour['date'] = per_hour['timestamp'].apply(lambda x: x.date())

In [ ]:
pivoted = per_hour.pivot_table(index='date', values='count', columns='hour').fillna(0)

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
ks = range(2, 20)

In [ ]:
models = [KMeans(k).fit(pivoted) for k in ks]

In [ ]:
scores = [-m.score(pivoted) for m in models]

In [ ]:
plt.plot(ks, scores)

In [ ]:
models[0].labels_.shape

In [ ]:
pivoted.shape

In [ ]:
dl = pandas.DataFrame({'date': pivoted.index.values, 'label': models[ks.index(5)].labels_})

In [ ]:
from calendar import day_abbr, month_abbr

In [ ]:
dl['weekday'] = dl.date.apply(lambda x: day_abbr[x.weekday()])

In [ ]:
dl['month'] = dl.date.apply(lambda x: month_abbr[x.month])

In [ ]:
s = dl.groupby(['label', 'weekday']).size()#.reset_index()#.pivot_table(index='label', values='0', columns='weekday')

In [ ]:
s.name = 'count'

In [ ]:
s = s.reset_index()

In [ ]:
s.pivot_table(index='label', columns='weekday', values='count').plot.bar()

In [ ]:
m = dl.groupby(['label', 'month']).size()

In [ ]:
m.name = 'count'

In [ ]:
m = m.reset_index()

In [ ]:
m.pivot_table(index='label', columns='month', values='count').plot.bar()
plt.legend(bbox_to_anchor=(1,1))

In [ ]:
for k, m in enumerate(models[ks.index(5)].cluster_centers_):
    plt.plot(m, label='{}'.format(k))
plt.legend()

In [ ]:
per_hour

In [ ]:
data.columns

In [ ]:
d = data[['timestamp', 'count', 'Start station']]

In [ ]:
per_time_and_station = d.groupby('Start station').apply(lambda x: x.set_index('timestamp').resample('1h').sum().fillna(0))

In [ ]:
del per_time_and_station['Start station']

In [ ]:
pts = per_time_and_station.reset_index()

In [ ]:
pts['date'] = pts.timestamp.apply(lambda x: x.date())

In [ ]:
pts['hour'] = pts.timestamp.apply(lambda x: x.hour)

In [ ]:
pivoted = pts.pivot_table(index=['date', 'Start station'], values='count', columns='hour').fillna(0)

In [ ]:
ks = range(2, 30)

In [ ]:
models = [KMeans(k).fit(pivoted) for k in ks]

In [ ]:
scores = [-m.score(pivoted) for m in models]

In [ ]:
plt.plot(ks, scores)

In [ ]:
for n, c in enumerate(models[ks.index(7)].cluster_centers_):
    plt.plot(c, label='{}'.format(n))
plt.legend()

In [ ]:
pred = pandas.DataFrame({'label': models[ks.index(7)].labels_}, index=pivoted.index).reset_index()

In [ ]:
pred.name

In [ ]:
p_new = pred.groupby(['label', 'Start station']).size().rename('count').reset_index()
#.pivot_table(index='label', columns='Start station', values='count').plot.bar()

In [ ]:
p_new[(p_new['count'] > 100) & (p_new.label == 3)]

In [ ]:
?pandas.read_json('data/bysykkel_stations.json', orient='records')

In [ ]:
import json

In [ ]:
s = json.dumps(json.load(open('data/bysykkel_stations.json'))['stations'])

In [ ]:
stations = pandas.read_json(s, orient='records')

In [ ]:
stations[stations.id == 245]